In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder, StandardScaler

# TODO:
# import black
# import jupyter_black
# jupyter_black.load(
#     lab=True,
#     line_length=100,
#     verbosity="INFO",
#     target_version=black.TargetVersion.PY310,
# )

In [2]:
# read the data   # TODO: function
path_to_train_data = "../data/train_file.xlsx"
df = pd.read_excel(path_to_train_data)
df.head() 

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,previous,poutcome,y
0,49,blue-collar,married,basic.9y,unknown,no,no,cellular,nov,wed,227,4,0,nonexistent,no
1,37,entrepreneur,married,university.degree,no,no,no,telephone,nov,wed,202,2,1,failure,no
2,78,retired,married,basic.4y,no,no,no,cellular,jul,mon,1148,1,0,nonexistent,yes
3,36,admin.,married,university.degree,no,yes,no,telephone,may,mon,120,2,0,nonexistent,no
4,59,retired,divorced,university.degree,no,no,no,cellular,jun,tue,368,2,0,nonexistent,no


In [3]:
df.drop_duplicates(keep="last", inplace=True)  # remove duplicate

#### Remove some features or their categories

The following *features* will be removed:
* **duration**: This feature is highly correlated with the dependent variable "y". The data suggest that longer contact times are associated with a higher probability of subscribing to a fixed-term deposit. However, the duration of a contact is only known after the contact has been completed and the customer has made his decision. If we want to use this model for predictive inference in production, where predictions need to be made before the contact takes place, including "duration" as a feature is impractical. Therefore, this feature should be excluded from the training data to ensure that the model can be used effectively for real-time prediction.
* **day_of_week**: EDA has shown that this feature does not have a significant impact on the customer"s decision. Given its minimal impact, including it as a feature would not significantly improve the predictive performance of the model. Removing this feature from the training data helps to simplify the model and focus on more important features.

In [4]:
features_to_remove = ["duration", "day_of_week"]
df_adjusted = df.drop(features_to_remove, axis=1)

**Dealing with unknown categories:** the *"unknown"* categories for such features, such as "job", "education", "default", "housing", "loan" will be removed, as they don't provide significant predictive value.

In [5]:
df_adjusted = df_adjusted.query('job != "unknown" & education != "unknown" & default != "unknown" & housing != "unknown"')

**Combining basic education categories:** to simplify the dataset and improve model performance, all basic education categories ("basic.4y", "basic.6y", "basic.9y") are combined into a single, more general category "education.basic". This will reduce the complexity of the education feature and help the model to generalize better by treating all levels of basic education as equivalent.

In [6]:
df_adjusted["education"] = df_adjusted["education"].replace(["basic.4y", "basic.6y", "basic.9y"], "education.basic")
df_adjusted.sample(n=3)

,age,job,marital,education,default,housing,loan,contact,month,campaign,previous,poutcome,y
19012,43,unemployed,divorced,education.basic,no,yes,yes,telephone,jul,2,0,nonexistent,no
27960,26,student,single,university.degree,no,yes,no,cellular,apr,4,0,nonexistent,no
19897,36,technician,single,university.degree,no,yes,no,cellular,may,1,1,failure,no


**Binning age:** given the wide distribution of ages in the dataset, we will split this category into four quantile-based bins. This approach will group the ages into four equally sized bins, which will help to normalize the distribution and potentially improve the performance of the model by reducing the effect of outliers.

In [7]:
bins_nmb = 4
age_order = ['young', 'young_adult', 'middle_aged', 'late_middle_aged']
bins_age = pd.qcut(df_adjusted["age"], q=4, labels=age_order)
df_adjusted.insert(1, "bins_age", bins_age) # Min/Max in each bin: [(16.999, 31.0] < (31.0, 37.0] < (37.0, 45.0] < (45.0, 91.0]]
# remove age column from dataframe
df_adjusted.drop("age", axis=1, inplace=True)

In [8]:
df_adjusted.bins_age.unique()

['young_adult', 'late_middle_aged', 'young', 'middle_aged']
Categories (4, object): ['young' < 'young_adult' < 'middle_aged' < 'late_middle_aged']

#### Encoding categorical features

In [9]:
# encoding ordinal data
# hierarchical order for bins_age
age_encoder = OrdinalEncoder(categories=[age_order])
df_adjusted["bins_age"] = age_encoder.fit_transform(df_adjusted[['bins_age']])

# education encoding
education_order = ['illiterate', 'education.basic', 'high.school', 'professional.course', 'university.degree']
education_encoder = OrdinalEncoder(categories=[education_order])
df_adjusted['education'] = education_encoder.fit_transform(df_adjusted[['education']])

# month encoding
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_encoder = OrdinalEncoder(categories=[month_order])
df_adjusted['month'] = month_encoder.fit_transform(df_adjusted[['month']])

# poutcome encoding
poutcome_order = ['nonexistent', 'failure', 'success']
poutcome_encoder = OrdinalEncoder(categories=[poutcome_order])
df_adjusted['poutcome'] = poutcome_encoder.fit_transform(df_adjusted[['poutcome']])

In [10]:
# encoding with LabelEncoder
label_encoder = LabelEncoder()
df_adjusted["contact"] = label_encoder.fit_transform(df_adjusted["contact"])

# encoding with binary values
mapping = {"yes": 1, "no": 0}
columns_to_map = ["default", "loan", "housing", "y"]
for column in columns_to_map:
    df_adjusted[column] = df_adjusted[column].map(mapping)

In [11]:
# encoding job and marital features with One-Hot Encoding
def one_hot_encode(df, df_feature):
    dummies = pd.get_dummies(df_feature, prefix=df_feature.name, dtype="int")
    return pd.concat([df, dummies], axis=1)

df_adjusted = one_hot_encode(df_adjusted, df_adjusted["job"])
df_adjusted = one_hot_encode(df_adjusted, df_adjusted["marital"])
df_adjusted.drop(["job", "marital"], axis=1, inplace=True)

In [12]:
df_adjusted.sample(n=5)

,bins_age,education,default,housing,loan,contact,month,campaign,previous,poutcome,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,marital_unknown
32024,1.0,4.0,0,1,0,0,8.0,1,0,0.0,...,0,0,0,0,0,0,0,1,0,0
18784,2.0,3.0,0,1,0,1,5.0,3,0,0.0,...,0,0,0,0,1,0,0,1,0,0
31087,2.0,2.0,0,1,0,0,6.0,1,0,0.0,...,0,0,0,0,0,0,0,1,0,0
22760,2.0,4.0,0,1,0,1,5.0,2,0,0.0,...,0,0,0,0,0,0,0,1,0,0
12916,3.0,1.0,0,0,0,0,4.0,2,0,0.0,...,0,0,0,0,0,0,0,1,0,0


#### Scaling of numeric and ordinal encoded features

In [13]:
scaler = StandardScaler()
features_to_scale = ['bins_age', 'previous', 'campaign', 'education', 'month']
df_adjusted[features_to_scale] = scaler.fit_transform(df_adjusted[features_to_scale])
df_adjusted.head()

,bins_age,education,default,housing,loan,contact,month,campaign,previous,poutcome,...,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,marital_divorced,marital_married,marital_single,marital_unknown
1,-0.406524,1.189200,0,0,0,1,2.010438,-0.189014,1.514811,1.0,...,0,0,0,0,0,0,0,1,0,0
2,1.376505,-1.293346,0,0,0,0,0.136531,-0.560054,-0.374158,0.0,...,1,0,0,0,0,0,0,1,0,0
3,-0.406524,1.189200,0,1,0,1,-0.800422,-0.189014,-0.374158,0.0,...,0,0,0,0,0,0,0,1,0,0
4,1.376505,1.189200,0,0,0,0,-0.331945,-0.189014,-0.374158,0.0,...,1,0,0,0,0,0,1,0,0,0
5,-1.298038,1.189200,0,0,0,0,0.605008,-0.189014,-0.374158,0.0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

# numeric_features = ["bins_age", "previous", "campaign"]
# fig, axs = plt.subplots(2, 2, figsize=(12, 6))
# axs = axs.flatten()
# for i, feature in enumerate(numeric_features):
#     sns.histplot(data=df_adjusted, x=feature, discrete=True, ax=axs[i], kde=True)
#     axs[i].grid(True)

# # to prevent overlap
# plt.tight_layout()